In [1]:
import torch
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

print("Device being used: %s" %device)

There are 1 GPU(s) available.
We will use the GPU: Tesla V100-SXM2-16GB
Device being used: cuda


In [2]:
path = '/scratch/kap9623/Final'
print(path)

/scratch/kap9623/Final


In [3]:
%%capture
!pip install simpletransformers

In [4]:
#retrieving paths for ann data

import glob
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from simpletransformers.ner import NERModel,NERArgs
# import wandb

ann_orig = glob.glob(path + '/ann_ref/*.ann' )
print(len(ann_orig))

389


# **Task 1: Name Entry Recognition for Medication Extraction**

# MODEL: ClinicalBERT

## Note-level Split

### Reformatting from BRAT to ConLL

In [5]:
outfile = path + '/ClinicalBERT/output_note_lvl.txt'
with open(outfile, 'w') as output_file:
  out_list = []
  for ann_file in ann_orig:
    with open(ann_file, 'r') as ann_in:
      txt_file = path + f'/ann_ref/{ann_file[-10:-4]}.txt'
      with open(txt_file, 'r') as txt_in:
        start_end = []
        
        #reading each line of .ann file and extracting start and end indices 
        for line in ann_in:
          entry = line.split()
          start_end.append([int(entry[2]), int(entry[3])])

        #reading each line of .txt file and extracting drug names using list of indices
        text_tokens = txt_in.read().split()
        drug_names = []
        for indicies in start_end:
          start = indicies[0]
          end = indicies[1]

          with open(txt_file, 'r') as txt_in:
            drug_names.append(txt_in.read()[indicies[0]:indicies[1]])
        
        #appending "O" or "DRUG" label to each txt token with relevant spacers
        for token in text_tokens:
          token_alnum = ''.join(ch for ch in token if ch.isalnum())
          if token_alnum in drug_names:
            if token == text_tokens[0]:
              out_list.append(f'{ann_file[-10:-4]}\t') #NOTE LEVEL
            else:
              out_list.append(f'{np.nan}\t')
            out_list.append(f'{token}\tDRUG\n')
          else:
            if token == text_tokens[0]:
              out_list.append(f'{ann_file[-10:-4]}\t') #NOTE LEVEL
            else:
              out_list.append(f'{np.nan}\t')
            out_list.append(f'{token}\tO\n')
        out_list.append('\n')
  out_string = ''.join(out_list)
  output_file.write(out_string)

### Reading ConLL output as dataframe

In [6]:
data = pd.read_csv(path + '/ClinicalBERT/output_note_lvl.txt', sep='\t', lineterminator='\n', names = ['Note', 'Token', 'Label'])
data = data.fillna(method = 'ffill')

In [7]:
#adding token count for each token by note
token_count = []
for note in data['Note'].unique():
  df = data.loc[data['Note']== note]
  count = 0
  for token in df['Token']:
    token_count.append(count)
    count += 1

data['Token Count'] = token_count
data.sort_values(by = ['Note', 'Token Count'], inplace=True)
data.reset_index(drop = True, inplace = True)

In [8]:
#label encoding Note column
le = LabelEncoder()
note_en = le.fit_transform(data['Note'])
data['Note'] = note_en

In [9]:
#renaming columns to match model requirements
data.rename(columns = {'Note':'sentence_id', 'Token':'words', 'Label':'labels'}, inplace=True)

### Model Building

In [10]:
#splitting data into featues and labels
X = data[['sentence_id','words']]
y = data['labels']

#splitting into train, val, test (70-10-20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle=False)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.125, shuffle=False)

#building up train data and test data
train_data = pd.DataFrame({"sentence_id":X_train["sentence_id"],"words":X_train["words"],"labels":y_train})
valid_data = pd.DataFrame({"sentence_id":X_valid["sentence_id"],"words":X_valid["words"],"labels":y_valid})
test_data = pd.DataFrame({"sentence_id":X_test["sentence_id"],"words":X_test["words"],"labels":y_test})

In [11]:
#checking unique token labels
label = data["labels"].unique().tolist()
label

['O', 'DRUG']

# Default performance (from previous run)

In [13]:
#setting hyperparameters to default for all models
args = NERArgs()
args.learning_rate = 1e-4
args.scheduler = 'constant_schedule'
args.overwrite_output_dir = True
args.manual_seed = 42
args.train_batch_size = 32
args.eval_batch_size = 32
args.num_train_epochs = 10

#instantiating model
model_cli_note = NERModel('bert', 'emilyalsentzer/Bio_ClinicalBERT',labels=label,args =args, use_cuda = True)

#training model on train data and tuning hyperparameters on valid data
model_cli_note.train_model(train_data,eval_data = valid_data)

# #reinstantiating NERModel using best epoch from training
# model_cli_note = NERModel('bert', 'outputs/best_model')

#evaluating on test data
mcn_result, mcn_outputs, mcn_preds = model_cli_note.eval_model(test_data)

print('Model: ClinicalBERT')
print('Evaluation metrics (Note-level split):')
mcn_result

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint 

  0%|          | 0/1 [00:00<?, ?it/s]

/home/kap9623/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/9 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/3 [00:00<?, ?it/s]

Model: ClinicalBERT
Evaluation metrics (Note-level split):


/home/kap9623/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DRUG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'eval_loss': 0.005606188516442974,
 'precision': 0.8936170212765957,
 'recall': 0.8936170212765957,
 'f1_score': 0.8936170212765957}

# Performance with freezing

In [16]:
#setting hyperparameters to default for all models
args = NERArgs()
args.learning_rate = 1e-4
args.scheduler = 'constant_schedule'
args.overwrite_output_dir = True
args.manual_seed = 42
args.train_batch_size = 32
args.eval_batch_size = 32
args.num_train_epochs = 10

#freezing embedding layers for pretrained model
# print(model_cli_note.get_named_parameters())
emb_layers_list = ['bert.embeddings.word_embeddings.weight', 'bert.embeddings.position_embeddings.weight', 'bert.embeddings.token_type_embeddings.weight', 'bert.embeddings.LayerNorm.weight', 'bert.embeddings.LayerNorm.bias']

#set lr for embedding layers to 0
args.custom_parameter_groups = [
    {
        "params": emb_layers_list,
        "lr": 0,
    }
]

#instantiating model
model_cli_note = NERModel('bert', 'emilyalsentzer/Bio_ClinicalBERT',labels=label,args =args, use_cuda = True)

#training model on train data and tuning hyperparameters on valid data
model_cli_note.train_model(train_data,eval_data = valid_data)

# #reinstantiating NERModel using best epoch from training
# model_cli_note = NERModel('bert', 'outputs/best_model')

#evaluating on test data
mcn_result, mcn_outputs, mcn_preds = model_cli_note.eval_model(test_data)

print('Model: ClinicalBERT')
print('Evaluation metrics (Note-level split):')
mcn_result

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint 

  0%|          | 0/1 [00:00<?, ?it/s]

/home/kap9623/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/9 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/3 [00:00<?, ?it/s]

Model: ClinicalBERT
Evaluation metrics (Note-level split):


/home/kap9623/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DRUG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'eval_loss': 0.011356407046453873,
 'precision': 0.9736842105263158,
 'recall': 0.7872340425531915,
 'f1_score': 0.8705882352941177}

# Performance with weighted loss

In [12]:
#calculating class weights based on imbalance
n_samples = len(train_data)
n_classes = 2
freq_O = train_data['labels'].value_counts()[0]
freq_DRUG = train_data['labels'].value_counts()[1]

weight_O = n_samples / (n_classes * freq_O) 
print(weight_O)

weight_DRUG = n_samples / (n_classes * freq_DRUG) 
print(weight_DRUG)

weights = [weight_O, weight_DRUG]

0.5125804829887884
20.372051035146846


In [19]:
#setting hyperparameters to default for all models
args = NERArgs()
args.learning_rate = 1e-4
args.scheduler = 'constant_schedule'
args.overwrite_output_dir = True
args.manual_seed = 42
args.train_batch_size = 32
args.eval_batch_size = 32
args.num_train_epochs = 10

#instantiating model
model_cli_note = NERModel('bert', 'emilyalsentzer/Bio_ClinicalBERT',labels=label,args =args, weight = weights, use_cuda = True)

#training model on train data and tuning hyperparameters on valid data
model_cli_note.train_model(train_data,eval_data = valid_data)

# #reinstantiating NERModel using best epoch from training
# model_cli_note = NERModel('bert', 'outputs/best_model')

#evaluating on test data
mcn_result, mcn_outputs, mcn_preds = model_cli_note.eval_model(test_data)

print('Model: ClinicalBERT')
print('Evaluation metrics (Note-level split):')
mcn_result

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint 

  0%|          | 0/1 [00:00<?, ?it/s]

/home/kap9623/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/9 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/3 [00:00<?, ?it/s]

Model: ClinicalBERT
Evaluation metrics (Note-level split):


/home/kap9623/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DRUG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'eval_loss': 0.13181999449928603,
 'precision': 0.8571428571428571,
 'recall': 0.8936170212765957,
 'f1_score': 0.875}

# Performance with freezing + weighted loss

In [20]:
#setting hyperparameters to default for all models
args = NERArgs()
args.learning_rate = 1e-4
args.scheduler = 'constant_schedule'
args.overwrite_output_dir = True
args.manual_seed = 42
args.train_batch_size = 32
args.eval_batch_size = 32
args.num_train_epochs = 10

#freezing embedding layers for pretrained model
# print(model_sci_note.get_named_parameters())
emb_layers_list = ['bert.embeddings.word_embeddings.weight', 'bert.embeddings.position_embeddings.weight', 'bert.embeddings.token_type_embeddings.weight', 'bert.embeddings.LayerNorm.weight', 'bert.embeddings.LayerNorm.bias']

#set lr for embedding layers to 0
args.custom_parameter_groups = [
    {
        "params": emb_layers_list,
        "lr": 0,
    }
]

#instantiating model
model_cli_note = NERModel('bert', 'emilyalsentzer/Bio_ClinicalBERT',labels=label,args =args, weight = weights, use_cuda = True)

#training model on train data and tuning hyperparameters on valid data
model_cli_note.train_model(train_data,eval_data = valid_data)

# #reinstantiating NERModel using best epoch from training
# model_cli_note = NERModel('bert', 'outputs/best_model')

#evaluating on test data
mcn_result, mcn_outputs, mcn_preds = model_cli_note.eval_model(test_data)

print('Model: ClinicalBERT')
print('Evaluation metrics (Note-level split):')
mcn_result

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint 

  0%|          | 0/1 [00:00<?, ?it/s]

/home/kap9623/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/9 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/3 [00:00<?, ?it/s]

Model: ClinicalBERT
Evaluation metrics (Note-level split):


/home/kap9623/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DRUG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'eval_loss': 0.1242379853501916,
 'precision': 0.8571428571428571,
 'recall': 0.8936170212765957,
 'f1_score': 0.875}

# Best Performance: No freezing + No weighted loss

# lr and bs tuning using best performance

### lr = 1e-4, bs = 8

In [21]:
#setting hyperparameters to default for all models
args = NERArgs()
args.learning_rate = 1e-4
args.scheduler = 'constant_schedule'
args.overwrite_output_dir = True
args.manual_seed = 42
args.train_batch_size = 8
args.eval_batch_size = 8
args.num_train_epochs = 10

#instantiating model
model_cli_note = NERModel('bert', 'emilyalsentzer/Bio_ClinicalBERT',labels=label,args =args, use_cuda = True)

#training model on train data and tuning hyperparameters on valid data
model_cli_note.train_model(train_data,eval_data = valid_data)

# #reinstantiating NERModel using best epoch from training
# model_cli_note = NERModel('bert', 'outputs/best_model')

#evaluating on test data
mcn_result, mcn_outputs, mcn_preds = model_cli_note.eval_model(test_data)

print('Model: ClinicalBERT')
print('Evaluation metrics (Note-level split):')
mcn_result

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint 

  0%|          | 0/1 [00:00<?, ?it/s]

/home/kap9623/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/34 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/10 [00:00<?, ?it/s]

Model: ClinicalBERT
Evaluation metrics (Note-level split):


/home/kap9623/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DRUG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'eval_loss': 0.0369361580349505,
 'precision': 0.9523809523809523,
 'recall': 0.425531914893617,
 'f1_score': 0.5882352941176471}

### lr = 1e-4, bs = 16

In [22]:
#setting hyperparameters to default for all models
args = NERArgs()
args.learning_rate = 1e-4
args.scheduler = 'constant_schedule'
args.overwrite_output_dir = True
args.manual_seed = 42
args.train_batch_size = 16
args.eval_batch_size = 16
args.num_train_epochs = 10

#instantiating model
model_cli_note = NERModel('bert', 'emilyalsentzer/Bio_ClinicalBERT',labels=label,args =args, use_cuda = True)

#training model on train data and tuning hyperparameters on valid data
model_cli_note.train_model(train_data,eval_data = valid_data)

# #reinstantiating NERModel using best epoch from training
# model_cli_note = NERModel('bert', 'outputs/best_model')

#evaluating on test data
mcn_result, mcn_outputs, mcn_preds = model_cli_note.eval_model(test_data)

print('Model: ClinicalBERT')
print('Evaluation metrics (Note-level split):')
mcn_result

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint 

  0%|          | 0/1 [00:00<?, ?it/s]

/home/kap9623/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/17 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/5 [00:00<?, ?it/s]

Model: ClinicalBERT
Evaluation metrics (Note-level split):


/home/kap9623/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DRUG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'eval_loss': 0.004918178968364373,
 'precision': 0.9545454545454546,
 'recall': 0.8936170212765957,
 'f1_score': 0.9230769230769231}

### lr = 1e-4, bs = 32

In [23]:
#setting hyperparameters to default for all models
args = NERArgs()
args.learning_rate = 1e-4
args.scheduler = 'constant_schedule'
args.overwrite_output_dir = True
args.manual_seed = 42
args.train_batch_size = 32
args.eval_batch_size = 32
args.num_train_epochs = 10

#instantiating model
model_cli_note = NERModel('bert', 'emilyalsentzer/Bio_ClinicalBERT',labels=label,args =args, use_cuda = True)

#training model on train data and tuning hyperparameters on valid data
model_cli_note.train_model(train_data,eval_data = valid_data)

# #reinstantiating NERModel using best epoch from training
# model_cli_note = NERModel('bert', 'outputs/best_model')

#evaluating on test data
mcn_result, mcn_outputs, mcn_preds = model_cli_note.eval_model(test_data)

print('Model: ClinicalBERT')
print('Evaluation metrics (Note-level split):')
mcn_result

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint 

  0%|          | 0/1 [00:00<?, ?it/s]

/home/kap9623/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/9 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/3 [00:00<?, ?it/s]

Model: ClinicalBERT
Evaluation metrics (Note-level split):


/home/kap9623/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DRUG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'eval_loss': 0.005606188516442974,
 'precision': 0.8936170212765957,
 'recall': 0.8936170212765957,
 'f1_score': 0.8936170212765957}

### lr = 1e-5, bs = 8

In [24]:
#setting hyperparameters to default for all models
args = NERArgs()
args.learning_rate = 1e-5
args.scheduler = 'constant_schedule'
args.overwrite_output_dir = True
args.manual_seed = 42
args.train_batch_size = 8
args.eval_batch_size = 8
args.num_train_epochs = 10

#instantiating model
model_cli_note = NERModel('bert', 'emilyalsentzer/Bio_ClinicalBERT',labels=label,args =args, use_cuda = True)

#training model on train data and tuning hyperparameters on valid data
model_cli_note.train_model(train_data,eval_data = valid_data)

# #reinstantiating NERModel using best epoch from training
# model_cli_note = NERModel('bert', 'outputs/best_model')

#evaluating on test data
mcn_result, mcn_outputs, mcn_preds = model_cli_note.eval_model(test_data)

print('Model: ClinicalBERT')
print('Evaluation metrics (Note-level split):')
mcn_result

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint 

  0%|          | 0/1 [00:00<?, ?it/s]

/home/kap9623/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/34 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/10 [00:00<?, ?it/s]

Model: ClinicalBERT
Evaluation metrics (Note-level split):


/home/kap9623/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DRUG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'eval_loss': 0.008717210425675147,
 'precision': 0.9090909090909091,
 'recall': 0.851063829787234,
 'f1_score': 0.8791208791208791}

### lr = 1e-5, bs = 16

In [25]:
#setting hyperparameters to default for all models
args = NERArgs()
args.learning_rate = 1e-5
args.scheduler = 'constant_schedule'
args.overwrite_output_dir = True
args.manual_seed = 42
args.train_batch_size = 16
args.eval_batch_size = 16
args.num_train_epochs = 10

#instantiating model
model_cli_note = NERModel('bert', 'emilyalsentzer/Bio_ClinicalBERT',labels=label,args =args, use_cuda = True)

#training model on train data and tuning hyperparameters on valid data
model_cli_note.train_model(train_data,eval_data = valid_data)

# #reinstantiating NERModel using best epoch from training
# model_cli_note = NERModel('bert', 'outputs/best_model')

#evaluating on test data
mcn_result, mcn_outputs, mcn_preds = model_cli_note.eval_model(test_data)

print('Model: ClinicalBERT')
print('Evaluation metrics (Note-level split):')
mcn_result

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint 

  0%|          | 0/1 [00:00<?, ?it/s]

/home/kap9623/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/17 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/5 [00:00<?, ?it/s]

Model: ClinicalBERT
Evaluation metrics (Note-level split):


/home/kap9623/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DRUG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'eval_loss': 0.00928337094374001,
 'precision': 0.926829268292683,
 'recall': 0.8085106382978723,
 'f1_score': 0.8636363636363636}

### lr = 1e-5, bs = 32

In [26]:
#setting hyperparameters to default for all models
args = NERArgs()
args.learning_rate = 1e-5
args.scheduler = 'constant_schedule'
args.overwrite_output_dir = True
args.manual_seed = 42
args.train_batch_size = 32
args.eval_batch_size = 32
args.num_train_epochs = 10

#instantiating model
model_cli_note = NERModel('bert', 'emilyalsentzer/Bio_ClinicalBERT',labels=label,args =args, use_cuda = True)

#training model on train data and tuning hyperparameters on valid data
model_cli_note.train_model(train_data,eval_data = valid_data)

# #reinstantiating NERModel using best epoch from training
# model_cli_note = NERModel('bert', 'outputs/best_model')

#evaluating on test data
mcn_result, mcn_outputs, mcn_preds = model_cli_note.eval_model(test_data)

print('Model: ClinicalBERT')
print('Evaluation metrics (Note-level split):')
mcn_result

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint 

  0%|          | 0/1 [00:00<?, ?it/s]

/home/kap9623/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/9 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/3 [00:00<?, ?it/s]

Model: ClinicalBERT
Evaluation metrics (Note-level split):


/home/kap9623/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DRUG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'eval_loss': 0.008269145851954818,
 'precision': 0.9024390243902439,
 'recall': 0.7872340425531915,
 'f1_score': 0.8409090909090909}

### lr = 1e-6, bs = 8

In [27]:
#setting hyperparameters to default for all models
args = NERArgs()
args.learning_rate = 1e-6
args.scheduler = 'constant_schedule'
args.overwrite_output_dir = True
args.manual_seed = 42
args.train_batch_size = 8
args.eval_batch_size = 8
args.num_train_epochs = 10

#instantiating model
model_cli_note = NERModel('bert', 'emilyalsentzer/Bio_ClinicalBERT',labels=label,args =args, use_cuda = True)

#training model on train data and tuning hyperparameters on valid data
model_cli_note.train_model(train_data,eval_data = valid_data)

# #reinstantiating NERModel using best epoch from training
# model_cli_note = NERModel('bert', 'outputs/best_model')

#evaluating on test data
mcn_result, mcn_outputs, mcn_preds = model_cli_note.eval_model(test_data)

print('Model: ClinicalBERT')
print('Evaluation metrics (Note-level split):')
mcn_result

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint 

  0%|          | 0/1 [00:00<?, ?it/s]

/home/kap9623/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/34 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/10 [00:00<?, ?it/s]

Model: ClinicalBERT
Evaluation metrics (Note-level split):


/home/kap9623/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DRUG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'eval_loss': 0.023241922492161393,
 'precision': 1.0,
 'recall': 0.23404255319148937,
 'f1_score': 0.3793103448275862}

### lr = 1e-6, bs = 16

In [28]:
#setting hyperparameters to default for all models
args = NERArgs()
args.learning_rate = 1e-6
args.scheduler = 'constant_schedule'
args.overwrite_output_dir = True
args.manual_seed = 42
args.train_batch_size = 16
args.eval_batch_size = 16
args.num_train_epochs = 10

#instantiating model
model_cli_note = NERModel('bert', 'emilyalsentzer/Bio_ClinicalBERT',labels=label,args =args, use_cuda = True)

#training model on train data and tuning hyperparameters on valid data
model_cli_note.train_model(train_data,eval_data = valid_data)

# #reinstantiating NERModel using best epoch from training
# model_cli_note = NERModel('bert', 'outputs/best_model')

#evaluating on test data
mcn_result, mcn_outputs, mcn_preds = model_cli_note.eval_model(test_data)

print('Model: ClinicalBERT')
print('Evaluation metrics (Note-level split):')
mcn_result

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint 

  0%|          | 0/1 [00:00<?, ?it/s]

/home/kap9623/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/17 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/5 [00:00<?, ?it/s]

Model: ClinicalBERT
Evaluation metrics (Note-level split):


/home/kap9623/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DRUG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/kap9623/.local/lib/python3.9/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 0.042488093674182895,
 'precision': 0.0,
 'recall': 0.0,
 'f1_score': 0.0}

### lr = 1e-6, bs = 32

In [29]:
#setting hyperparameters to default for all models
args = NERArgs()
args.learning_rate = 1e-6
args.scheduler = 'constant_schedule'
args.overwrite_output_dir = True
args.manual_seed = 42
args.train_batch_size = 32
args.eval_batch_size = 32
args.num_train_epochs = 10

#instantiating model
model_cli_note = NERModel('bert', 'emilyalsentzer/Bio_ClinicalBERT',labels=label,args =args, use_cuda = True)

#training model on train data and tuning hyperparameters on valid data
model_cli_note.train_model(train_data,eval_data = valid_data)

# #reinstantiating NERModel using best epoch from training
# model_cli_note = NERModel('bert', 'outputs/best_model')

#evaluating on test data
mcn_result, mcn_outputs, mcn_preds = model_cli_note.eval_model(test_data)

print('Model: ClinicalBERT')
print('Evaluation metrics (Note-level split):')
mcn_result

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint 

  0%|          | 0/1 [00:00<?, ?it/s]

/home/kap9623/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/9 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/3 [00:00<?, ?it/s]

Model: ClinicalBERT
Evaluation metrics (Note-level split):


/home/kap9623/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DRUG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'eval_loss': 0.22048310935497284,
 'precision': 0.0,
 'recall': 0.0,
 'f1_score': 0.0}

### lr = 5e-4, bs = 8

In [30]:
#setting hyperparameters to default for all models
args = NERArgs()
args.learning_rate = 5e-4
args.scheduler = 'constant_schedule'
args.overwrite_output_dir = True
args.manual_seed = 42
args.train_batch_size = 8
args.eval_batch_size = 8
args.num_train_epochs = 10

#instantiating model
model_cli_note = NERModel('bert', 'emilyalsentzer/Bio_ClinicalBERT',labels=label,args =args, use_cuda = True)

#training model on train data and tuning hyperparameters on valid data
model_cli_note.train_model(train_data,eval_data = valid_data)

# #reinstantiating NERModel using best epoch from training
# model_cli_note = NERModel('bert', 'outputs/best_model')

#evaluating on test data
mcn_result, mcn_outputs, mcn_preds = model_cli_note.eval_model(test_data)

print('Model: ClinicalBERT')
print('Evaluation metrics (Note-level split):')
mcn_result

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint 

  0%|          | 0/1 [00:00<?, ?it/s]

/home/kap9623/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/34 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/10 [00:00<?, ?it/s]

Model: ClinicalBERT
Evaluation metrics (Note-level split):


/home/kap9623/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DRUG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/kap9623/.local/lib/python3.9/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 0.05526755172759294,
 'precision': 0.0,
 'recall': 0.0,
 'f1_score': 0.0}

### lr = 5e-4, bs = 16

In [31]:
#setting hyperparameters to default for all models
args = NERArgs()
args.learning_rate = 5e-4
args.scheduler = 'constant_schedule'
args.overwrite_output_dir = True
args.manual_seed = 42
args.train_batch_size = 16
args.eval_batch_size = 16
args.num_train_epochs = 10

#instantiating model
model_cli_note = NERModel('bert', 'emilyalsentzer/Bio_ClinicalBERT',labels=label,args =args, use_cuda = True)

#training model on train data and tuning hyperparameters on valid data
model_cli_note.train_model(train_data,eval_data = valid_data)

# #reinstantiating NERModel using best epoch from training
# model_cli_note = NERModel('bert', 'outputs/best_model')

#evaluating on test data
mcn_result, mcn_outputs, mcn_preds = model_cli_note.eval_model(test_data)

print('Model: ClinicalBERT')
print('Evaluation metrics (Note-level split):')
mcn_result

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint 

  0%|          | 0/1 [00:00<?, ?it/s]

/home/kap9623/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/17 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/5 [00:00<?, ?it/s]

Model: ClinicalBERT
Evaluation metrics (Note-level split):


/home/kap9623/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DRUG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/kap9623/.local/lib/python3.9/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 0.05702500082552433,
 'precision': 0.0,
 'recall': 0.0,
 'f1_score': 0.0}

### lr = 5e-4, bs = 32

In [32]:
#setting hyperparameters to default for all models
args = NERArgs()
args.learning_rate = 5e-4
args.scheduler = 'constant_schedule'
args.overwrite_output_dir = True
args.manual_seed = 42
args.train_batch_size = 32
args.eval_batch_size = 32
args.num_train_epochs = 10

#instantiating model
model_cli_note = NERModel('bert', 'emilyalsentzer/Bio_ClinicalBERT',labels=label,args =args, use_cuda = True)

#training model on train data and tuning hyperparameters on valid data
model_cli_note.train_model(train_data,eval_data = valid_data)

# #reinstantiating NERModel using best epoch from training
# model_cli_note = NERModel('bert', 'outputs/best_model')

#evaluating on test data
mcn_result, mcn_outputs, mcn_preds = model_cli_note.eval_model(test_data)

print('Model: ClinicalBERT')
print('Evaluation metrics (Note-level split):')
mcn_result

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint 

  0%|          | 0/1 [00:00<?, ?it/s]

/home/kap9623/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/9 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/3 [00:00<?, ?it/s]

Model: ClinicalBERT
Evaluation metrics (Note-level split):


/home/kap9623/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DRUG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'eval_loss': 0.03243309942384561,
 'precision': 0.5625,
 'recall': 0.574468085106383,
 'f1_score': 0.5684210526315789}

### lr = 5e-5, bs = 8

In [33]:
#setting hyperparameters to default for all models
args = NERArgs()
args.learning_rate = 5e-5
args.scheduler = 'constant_schedule'
args.overwrite_output_dir = True
args.manual_seed = 42
args.train_batch_size = 8
args.eval_batch_size = 8
args.num_train_epochs = 10

#instantiating model
model_cli_note = NERModel('bert', 'emilyalsentzer/Bio_ClinicalBERT',labels=label,args =args, use_cuda = True)

#training model on train data and tuning hyperparameters on valid data
model_cli_note.train_model(train_data,eval_data = valid_data)

# #reinstantiating NERModel using best epoch from training
# model_cli_note = NERModel('bert', 'outputs/best_model')

#evaluating on test data
mcn_result, mcn_outputs, mcn_preds = model_cli_note.eval_model(test_data)

print('Model: ClinicalBERT')
print('Evaluation metrics (Note-level split):')
mcn_result

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint 

  0%|          | 0/1 [00:00<?, ?it/s]

/home/kap9623/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/34 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/10 [00:00<?, ?it/s]

Model: ClinicalBERT
Evaluation metrics (Note-level split):


/home/kap9623/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DRUG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'eval_loss': 0.011076752160352043,
 'precision': 0.95,
 'recall': 0.8085106382978723,
 'f1_score': 0.8735632183908046}

### lr = 5e-5, bs = 16

In [34]:
#setting hyperparameters to default for all models
args = NERArgs()
args.learning_rate = 5e-5
args.scheduler = 'constant_schedule'
args.overwrite_output_dir = True
args.manual_seed = 42
args.train_batch_size = 16
args.eval_batch_size = 16
args.num_train_epochs = 10

#instantiating model
model_cli_note = NERModel('bert', 'emilyalsentzer/Bio_ClinicalBERT',labels=label,args =args, use_cuda = True)

#training model on train data and tuning hyperparameters on valid data
model_cli_note.train_model(train_data,eval_data = valid_data)

# #reinstantiating NERModel using best epoch from training
# model_cli_note = NERModel('bert', 'outputs/best_model')

#evaluating on test data
mcn_result, mcn_outputs, mcn_preds = model_cli_note.eval_model(test_data)

print('Model: ClinicalBERT')
print('Evaluation metrics (Note-level split):')
mcn_result

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint 

  0%|          | 0/1 [00:00<?, ?it/s]

/home/kap9623/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/17 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/5 [00:00<?, ?it/s]

Model: ClinicalBERT
Evaluation metrics (Note-level split):


/home/kap9623/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DRUG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'eval_loss': 0.018453283049166203,
 'precision': 0.9705882352941176,
 'recall': 0.7021276595744681,
 'f1_score': 0.8148148148148149}

### lr = 5e-5, bs = 32

In [35]:
#setting hyperparameters to default for all models
args = NERArgs()
args.learning_rate = 5e-5
args.scheduler = 'constant_schedule'
args.overwrite_output_dir = True
args.manual_seed = 42
args.train_batch_size = 32
args.eval_batch_size = 32
args.num_train_epochs = 10

#instantiating model
model_cli_note = NERModel('bert', 'emilyalsentzer/Bio_ClinicalBERT',labels=label,args =args, use_cuda = True)

#training model on train data and tuning hyperparameters on valid data
model_cli_note.train_model(train_data,eval_data = valid_data)

# #reinstantiating NERModel using best epoch from training
# model_cli_note = NERModel('bert', 'outputs/best_model')

#evaluating on test data
mcn_result, mcn_outputs, mcn_preds = model_cli_note.eval_model(test_data)

print('Model: ClinicalBERT')
print('Evaluation metrics (Note-level split):')
mcn_result

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint 

  0%|          | 0/1 [00:00<?, ?it/s]

/home/kap9623/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/9 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/3 [00:00<?, ?it/s]

Model: ClinicalBERT
Evaluation metrics (Note-level split):


/home/kap9623/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DRUG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'eval_loss': 0.005753227402844156,
 'precision': 0.9761904761904762,
 'recall': 0.8723404255319149,
 'f1_score': 0.9213483146067415}

### lr = 5e-6, bs = 8

In [36]:
#setting hyperparameters to default for all models
args = NERArgs()
args.learning_rate = 5e-6
args.scheduler = 'constant_schedule'
args.overwrite_output_dir = True
args.manual_seed = 42
args.train_batch_size = 8
args.eval_batch_size = 8
args.num_train_epochs = 10

#instantiating model
model_cli_note = NERModel('bert', 'emilyalsentzer/Bio_ClinicalBERT',labels=label,args =args, use_cuda = True)

#training model on train data and tuning hyperparameters on valid data
model_cli_note.train_model(train_data,eval_data = valid_data)

# #reinstantiating NERModel using best epoch from training
# model_cli_note = NERModel('bert', 'outputs/best_model')

#evaluating on test data
mcn_result, mcn_outputs, mcn_preds = model_cli_note.eval_model(test_data)

print('Model: ClinicalBERT')
print('Evaluation metrics (Note-level split):')
mcn_result

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint 

  0%|          | 0/1 [00:00<?, ?it/s]

/home/kap9623/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/34 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/10 [00:00<?, ?it/s]

Model: ClinicalBERT
Evaluation metrics (Note-level split):


/home/kap9623/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DRUG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'eval_loss': 0.008412482670973987,
 'precision': 0.8837209302325582,
 'recall': 0.8085106382978723,
 'f1_score': 0.8444444444444444}

### lr = 5e-6, bs = 16

In [37]:
#setting hyperparameters to default for all models
args = NERArgs()
args.learning_rate = 5e-6
args.scheduler = 'constant_schedule'
args.overwrite_output_dir = True
args.manual_seed = 42
args.train_batch_size = 16
args.eval_batch_size = 16
args.num_train_epochs = 10

#instantiating model
model_cli_note = NERModel('bert', 'emilyalsentzer/Bio_ClinicalBERT',labels=label,args =args, use_cuda = True)

#training model on train data and tuning hyperparameters on valid data
model_cli_note.train_model(train_data,eval_data = valid_data)

# #reinstantiating NERModel using best epoch from training
# model_cli_note = NERModel('bert', 'outputs/best_model')

#evaluating on test data
mcn_result, mcn_outputs, mcn_preds = model_cli_note.eval_model(test_data)

print('Model: ClinicalBERT')
print('Evaluation metrics (Note-level split):')
mcn_result

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint 

  0%|          | 0/1 [00:00<?, ?it/s]

/home/kap9623/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/17 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/5 [00:00<?, ?it/s]

Model: ClinicalBERT
Evaluation metrics (Note-level split):


/home/kap9623/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DRUG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'eval_loss': 0.009532946487888693,
 'precision': 0.9487179487179487,
 'recall': 0.7872340425531915,
 'f1_score': 0.8604651162790696}

### lr = 5e-6, bs = 32

In [38]:
#setting hyperparameters to default for all models
args = NERArgs()
args.learning_rate = 5e-6
args.scheduler = 'constant_schedule'
args.overwrite_output_dir = True
args.manual_seed = 42
args.train_batch_size = 32
args.eval_batch_size = 32
args.num_train_epochs = 10

#instantiating model
model_cli_note = NERModel('bert', 'emilyalsentzer/Bio_ClinicalBERT',labels=label,args =args, use_cuda = True)

#training model on train data and tuning hyperparameters on valid data
model_cli_note.train_model(train_data,eval_data = valid_data)

# #reinstantiating NERModel using best epoch from training
# model_cli_note = NERModel('bert', 'outputs/best_model')

#evaluating on test data
mcn_result, mcn_outputs, mcn_preds = model_cli_note.eval_model(test_data)

print('Model: ClinicalBERT')
print('Evaluation metrics (Note-level split):')
mcn_result

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint 

  0%|          | 0/1 [00:00<?, ?it/s]

/home/kap9623/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/9 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/3 [00:00<?, ?it/s]

Model: ClinicalBERT
Evaluation metrics (Note-level split):


/home/kap9623/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DRUG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'eval_loss': 0.014987803529947996,
 'precision': 0.9642857142857143,
 'recall': 0.574468085106383,
 'f1_score': 0.7200000000000001}

# Best Performance: lr = 1e-4, bs = 16

# Checking performance without and with early stopping (25 epochs)

In [13]:
#WITHOUT EARLY STOPPING

#setting hyperparameters to default for all models
args = NERArgs()
args.learning_rate = 1e-4
args.scheduler = 'constant_schedule'
args.overwrite_output_dir = True
args.manual_seed = 42
args.train_batch_size = 16
args.eval_batch_size = 16
args.num_train_epochs = 25

#instantiating model
model_cli_note = NERModel('bert', 'emilyalsentzer/Bio_ClinicalBERT',labels=label,args =args, use_cuda = True)

#training model on train data and tuning hyperparameters on valid data
model_cli_note.train_model(train_data,eval_data = valid_data)

# #reinstantiating NERModel using best epoch from training
# model_cli_note = NERModel('bert', 'outputs/best_model')

#evaluating on test data
mcn_result, mcn_outputs, mcn_preds = model_cli_note.eval_model(test_data)

print('Model: ClinicalBERT')
print('Evaluation metrics (Note-level split):')
mcn_result

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint 

  0%|          | 0/1 [00:00<?, ?it/s]

/home/kap9623/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/25 [00:00<?, ?it/s]

Running Epoch 0 of 25:   0%|          | 0/17 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/5 [00:00<?, ?it/s]

Model: ClinicalBERT
Evaluation metrics (Note-level split):


/home/kap9623/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DRUG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'eval_loss': 0.029252386093139647,
 'precision': 0.9655172413793104,
 'recall': 0.5957446808510638,
 'f1_score': 0.7368421052631579}

In [15]:
# Num steps in epoch = num training samples / batch size
steps_per_epoch = int(np.ceil(len(train_data) / float(args.train_batch_size)))

print('Each epoch will have {:,} steps.'.format(steps_per_epoch))

Each epoch will have 10,579 steps.


In [14]:
#WITH EARLY STOPPING

#setting hyperparameters to default for all models
args = NERArgs()
args.learning_rate = 1e-4
args.scheduler = 'constant_schedule'
args.overwrite_output_dir = True
args.manual_seed = 42
args.train_batch_size = 16
args.eval_batch_size = 16
args.num_train_epochs = 25

# Turn on early stopping
args.evaluate_during_training = True
args.evaluate_during_training_steps = 2116 #evaluating 10 times every epochs
args.use_cached_eval_features = True
args.use_early_stopping = True
args.early_stopping_delta = 0.01
args.early_stopping_metric = "eval_loss"
args.early_stopping_metric_minimize = True
args.early_stopping_patience = 2

#instantiating model
model_cli_note = NERModel('bert', 'emilyalsentzer/Bio_ClinicalBERT',labels=label,args =args, use_cuda = True)

#training model on train data and tuning hyperparameters on valid data
model_cli_note.train_model(train_data,eval_data = valid_data)

# #reinstantiating NERModel using best epoch from training
# model_cli_note = NERModel('bert', 'outputs/best_model')

#evaluating on test data
mcn_result, mcn_outputs, mcn_preds = model_cli_note.eval_model(test_data)

print('Model: ClinicalBERT')
print('Evaluation metrics (Note-level split):')
mcn_result

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint 

  0%|          | 0/1 [00:00<?, ?it/s]

/home/kap9623/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/25 [00:00<?, ?it/s]

Running Epoch 0 of 25:   0%|          | 0/17 [00:00<?, ?it/s]

...

/home/kap9623/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DRUG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


Running Evaluation:   0%|          | 0/5 [00:00<?, ?it/s]

Model: ClinicalBERT
Evaluation metrics (Note-level split):


/home/kap9623/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DRUG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'eval_loss': 0.007674673572182655,
 'precision': 0.9736842105263158,
 'recall': 0.7872340425531915,
 'f1_score': 0.8705882352941177}